# Loop 14 Analysis: Strategic Assessment

## Current Status
- Best score: 70.659475 (multi-source ensemble)
- Target: 68.919154
- Gap: 1.740321 points (2.53%)

## Key Questions
1. What is the theoretical minimum score?
2. Where are the biggest opportunities for improvement?
3. What approaches haven't been tried?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Load current best submission
df = pd.read_csv('/home/submission/submission.csv')
print(f"Total rows: {len(df)}")
print(df.head())

In [ ]:
# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

def get_tree_poly(x, y, deg):
    coords = [
        (0.0, TIP_Y), (TOP_W / 2.0, TIER_1_Y), (TOP_W / 4.0, TIER_1_Y),
        (MID_W / 2.0, TIER_2_Y), (MID_W / 4.0, TIER_2_Y), (BASE_W / 2.0, BASE_Y),
        (TRUNK_W / 2.0, BASE_Y), (TRUNK_W / 2.0, TRUNK_BOTTOM_Y),
        (-TRUNK_W / 2.0, TRUNK_BOTTOM_Y), (-TRUNK_W / 2.0, BASE_Y),
        (-BASE_W / 2.0, BASE_Y), (-MID_W / 4.0, TIER_2_Y), (-MID_W / 2.0, TIER_2_Y),
        (-TOP_W / 4.0, TIER_1_Y), (-TOP_W / 2.0, TIER_1_Y),
    ]
    poly = Polygon(coords)
    return affinity.translate(affinity.rotate(poly, deg, origin=(0, 0)), x, y)

# Calculate tree area
tree = get_tree_poly(0, 0, 0)
tree_area = tree.area
print(f"Tree area: {tree_area:.6f}")
print(f"Tree bounding box at 0 deg: {tree.bounds}")

# Calculate bounding box dimensions at different angles
for angle in [0, 45, 90, 135]:
    t = get_tree_poly(0, 0, angle)
    bounds = t.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    print(f"Angle {angle}: width={width:.4f}, height={height:.4f}, bbox_area={width*height:.4f}")

In [ ]:
# Calculate score breakdown by N
def calculate_score_for_n(df, n):
    prefix = f"{n:03d}_"
    group = df[df['id'].str.startswith(prefix)].sort_values('id')
    if len(group) != n:
        return None
    
    xs = group['x'].values.astype(float)
    ys = group['y'].values.astype(float)
    degs = group['angle'].values.astype(float)
    
    min_x = min_y = 1e10
    max_x = max_y = -1e10
    for i in range(n):
        poly = get_tree_poly(xs[i], ys[i], degs[i])
        bounds = poly.bounds
        if bounds[0] < min_x: min_x = bounds[0]
        if bounds[1] < min_y: min_y = bounds[1]
        if bounds[2] > max_x: max_x = bounds[2]
        if bounds[3] > max_y: max_y = bounds[3]
    
    side = max(max_x - min_x, max_y - min_y)
    return side * side / n

# Calculate scores for all N
scores = {}
for n in range(1, 201):
    score = calculate_score_for_n(df, n)
    if score:
        scores[n] = score

print(f"Total score: {sum(scores.values()):.6f}")
print(f"\nTop 10 contributors to score:")
for n, s in sorted(scores.items(), key=lambda x: -x[1])[:10]:
    print(f"  N={n}: {s:.6f} ({s/sum(scores.values())*100:.2f}%)")

print(f"\nBottom 10 contributors to score:")
for n, s in sorted(scores.items(), key=lambda x: x[1])[:10]:
    print(f"  N={n}: {s:.6f} ({s/sum(scores.values())*100:.2f}%)")

In [ ]:
# Theoretical minimum analysis
# For N trees, the minimum bounding box area is N * tree_area (if perfect packing)
# The score is side^2 / N, so minimum score is tree_area (if perfect square packing)

print("Theoretical minimum analysis:")
print(f"Tree area: {tree_area:.6f}")
print(f"If perfect packing (100% efficiency), score per N = tree_area = {tree_area:.6f}")
print(f"Theoretical minimum total score (200 N values): {tree_area * 200:.6f}")

# Calculate packing efficiency for each N
print("\nPacking efficiency by N:")
efficiencies = {}
for n in [1, 2, 5, 10, 20, 50, 100, 150, 200]:
    if n in scores:
        # score = side^2 / n, so side^2 = score * n
        # packing efficiency = (n * tree_area) / side^2 = (n * tree_area) / (score * n) = tree_area / score
        efficiency = tree_area / scores[n]
        efficiencies[n] = efficiency
        print(f"  N={n}: score={scores[n]:.6f}, efficiency={efficiency*100:.2f}%")

In [ ]:
# Analyze the gap to target
target = 68.919154
current = sum(scores.values())
gap = current - target

print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")

# How much improvement needed per N on average?
print(f"\nAverage improvement needed per N: {gap/200:.6f}")

# Which N values have the most room for improvement?
print("\nN values with worst packing efficiency (most room for improvement):")
for n, eff in sorted(efficiencies.items(), key=lambda x: x[1])[:10]:
    potential_improvement = scores[n] - tree_area
    print(f"  N={n}: efficiency={eff*100:.2f}%, potential improvement={potential_improvement:.6f}")

In [ ]:
# Analyze small N values in detail
print("Detailed analysis of small N values:")
for n in range(1, 11):
    if n in scores:
        # Get the configuration
        prefix = f"{n:03d}_"
        group = df[df['id'].str.startswith(prefix)].sort_values('id')
        xs = group['x'].values.astype(float)
        ys = group['y'].values.astype(float)
        degs = group['angle'].values.astype(float)
        
        # Calculate bounding box
        min_x = min_y = 1e10
        max_x = max_y = -1e10
        for i in range(n):
            poly = get_tree_poly(xs[i], ys[i], degs[i])
            bounds = poly.bounds
            if bounds[0] < min_x: min_x = bounds[0]
            if bounds[1] < min_y: min_y = bounds[1]
            if bounds[2] > max_x: max_x = bounds[2]
            if bounds[3] > max_y: max_y = bounds[3]
        
        width = max_x - min_x
        height = max_y - min_y
        side = max(width, height)
        
        print(f"\nN={n}:")
        print(f"  Score: {scores[n]:.6f}")
        print(f"  Bounding box: {width:.4f} x {height:.4f}")
        print(f"  Side length: {side:.4f}")
        print(f"  Angles: {degs}")
        print(f"  Efficiency: {tree_area/scores[n]*100:.2f}%")

In [ ]:
# Check N=1 specifically - what is the optimal angle?
print("N=1 Analysis:")
print("Testing different angles for N=1:")

best_score = float('inf')
best_angle = None

for angle in range(0, 360, 1):
    poly = get_tree_poly(0, 0, angle)
    bounds = poly.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    side = max(width, height)
    score = side * side / 1
    
    if score < best_score:
        best_score = score
        best_angle = angle

print(f"Best angle for N=1: {best_angle} degrees")
print(f"Best score for N=1: {best_score:.6f}")
print(f"Current N=1 score: {scores[1]:.6f}")
print(f"Potential improvement: {scores[1] - best_score:.6f}")

# More precise search around best angle
print("\nFine-grained search around best angle:")
for angle in np.arange(best_angle - 5, best_angle + 5, 0.1):
    poly = get_tree_poly(0, 0, angle)
    bounds = poly.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    side = max(width, height)
    score = side * side / 1
    
    if score < best_score:
        best_score = score
        best_angle = angle

print(f"Best angle for N=1 (fine): {best_angle:.1f} degrees")
print(f"Best score for N=1 (fine): {best_score:.6f}")

In [ ]:
# Summary of findings
print("="*60)
print("SUMMARY OF FINDINGS")
print("="*60)
print(f"\n1. Current best score: {current:.6f}")
print(f"2. Target score: {target:.6f}")
print(f"3. Gap: {gap:.6f} ({gap/current*100:.2f}%)")
print(f"\n4. Tree area: {tree_area:.6f}")
print(f"5. Theoretical minimum (perfect packing): {tree_area * 200:.6f}")
print(f"6. Current efficiency: {(tree_area * 200) / current * 100:.2f}%")
print(f"7. Target efficiency: {(tree_area * 200) / target * 100:.2f}%")

print(f"\n8. Small N values (1-10) contribute: {sum(scores[n] for n in range(1,11)):.6f} ({sum(scores[n] for n in range(1,11))/current*100:.2f}%)")
print(f"9. Large N values (100-200) contribute: {sum(scores[n] for n in range(100,201)):.6f} ({sum(scores[n] for n in range(100,201))/current*100:.2f}%)")

print("\n" + "="*60)
print("KEY INSIGHT: The gap to target (1.74 points) is 2.53% of current score.")
print("This requires either:")
print("  a) Finding 2.53% improvement across all N values")
print("  b) Finding large improvements in specific N values")
print("  c) Using techniques not available in public kernels")
print("="*60)